In [55]:
import scda
import pprint

# Demo 1: IrisAO APLC

## Set some design parameters

In [40]:
pupil_params = {'N': 200}
fpm_params = {'rad': 9.898/2, 'M':50}
ls_params = {}
image_params = {'c': 10., 'iwa':3.5, 'owa':10.}
design_params = {'Pupil': pupil_params, 'FPM': fpm_params, 'LS': ls_params, 'Image': image_params}

## file organization

In [41]:
test_dir = "test_aplc_wrapper" # a nominal destination for new AMPL programs
aux_dir = "~/SCDA/2d AMPL script - half pupil" # location of input masks for AMPL program

fileorg = {'work dir': test_dir, 'TelAp dir': aux_dir, 'FPM dir': aux_dir, 'LS dir': aux_dir, \
           'TelAp fname': "IRISAO_N=0200_center_half_spiders3=01_gapID=10_BW.dat", \
           'FPM fname': "CircPupil_N=0050_obs=00_center_half.dat", \
           'LS fname': "IRISAO-0_N=0200_center_half_spiders3=02_ID=20_OD=098.dat"}

## Options for constraints and optimizer

In [42]:
solver_params = {'method': 'bar', 'presolve': False, 'Nthreads': 8}

## Create a coronagraph design object, then write its AMPL source code file.

In [51]:
irisao_coron = scda.HalfplaneAPLC( fileorg=fileorg, design=design_params, solver=solver_params )
irisao_coron.write_ampl(ampl_src_fname="test_aplc.mod", overwrite=True)

# Demo 2: SCDA paramater survey 

## Define a list of pupil parameters to survey
Parameters selected here are the primary mirror (pm), support struts (ss), secondary strut thickness (sst), secondary obstruction (so).
The last parameter, 'N' = pupil array diameter, has only one value and therefore is fixed for all designs. 

In [52]:
jpl_telap_all = {'Pupil': { 'pm': ['hex1', 'hex2', 'hex3', 'key24', 'pie12', 'pie8'],
                            'ss': ['y60','y60off','x','cross','t','y90'],
                            'sst': ['025','100'],
                            'so': [True, False],
                            'N': 125 } }

In [53]:
survey_ampl_dir = "./ampl_survey_test/" # where to write the AMPL source code

## Initiate a design parameter survey object with the above parameter combinations 

In [46]:
telap_survey = scda.DesignParamSurvey(scda.HalfplaneAPLC, jpl_telap_all, fileorg={'ampl src dir':survey_ampl_dir})
print("This survey has {0:d} design parameter combinations.".format(len(telap_survey.coron_list)))
print("{0:d} parameters are varied: {1}".format(len(telap_survey.varied_param_index), telap_survey.varied_param_index))

This survey has 144 design parameter combinations.
4 parameters are varied: (('Pupil', 'pm'), ('Pupil', 'ss'), ('Pupil', 'sst'), ('Pupil', 'so'))


## Print some attributes of an individual coronagraph object

In [47]:
i = 50
print("Telescope aperture file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['TelAp fname']))
print("Focal plane mask file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['FPM fname']))
print("Lyot stop file for design #{:d}: {:s}".format(i+1, telap_survey.coron_list[i].fileorg['LS fname']))

Telescope aperture file for design #51: /Users/ntz/SCDA/scda_pytools/TelAp_half_hex3y60t100so1_N0125.dat
Focal plane mask file for design #51: /Users/ntz/SCDA/scda_pytools/FPM_occspot_M050.dat
Lyot stop file for design #51: /Users/ntz/SCDA/scda_pytools/LS_half_hex3y60t100so1_N0125_20D90ovsz00.dat


## List the varying parameter combinations

In [48]:
pprint.pprint(telap_survey.varied_param_combos)

(('hex1', 'y60', '025', True),
 ('hex1', 'y60', '025', False),
 ('hex1', 'y60', '100', True),
 ('hex1', 'y60', '100', False),
 ('hex1', 'y60off', '025', True),
 ('hex1', 'y60off', '025', False),
 ('hex1', 'y60off', '100', True),
 ('hex1', 'y60off', '100', False),
 ('hex1', 'x', '025', True),
 ('hex1', 'x', '025', False),
 ('hex1', 'x', '100', True),
 ('hex1', 'x', '100', False),
 ('hex1', 'cross', '025', True),
 ('hex1', 'cross', '025', False),
 ('hex1', 'cross', '100', True),
 ('hex1', 'cross', '100', False),
 ('hex1', 't', '025', True),
 ('hex1', 't', '025', False),
 ('hex1', 't', '100', True),
 ('hex1', 't', '100', False),
 ('hex1', 'y90', '025', True),
 ('hex1', 'y90', '025', False),
 ('hex1', 'y90', '100', True),
 ('hex1', 'y90', '100', False),
 ('hex2', 'y60', '025', True),
 ('hex2', 'y60', '025', False),
 ('hex2', 'y60', '100', True),
 ('hex2', 'y60', '100', False),
 ('hex2', 'y60off', '025', True),
 ('hex2', 'y60off', '025', False),
 ('hex2', 'y60off', '100', True),
 ('hex2', '

## Write the batch of AMPL files

In [54]:
telap_survey.write_ampl(override_infile_status=True, overwrite=True)